# Quantify Cell Migration

In [9]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

This notebook accompanies version 2 of the publication 

> Juarez EF, Garri C, Ghaffarizadeh A et al. Quantification of cancer cell migration with an integrated experimental-computational pipeline [version 1; peer review: 2 approved with reservations]. F1000Research 2018, 7:1296
(https://doi.org/10.12688/f1000research.15599.1)

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
    
- When you see a blue box like this one, follow those instructions to run this notebook successfully.  
- Run through the each of the 3 steps outlined in this notebook in order.
</div>

# Technical Requirements

<div class="well well-sm">
<ul>
<li> This notebook uses the standard GPNB docker container taged "latest" on 2019-04-01  </li>
<li> Using the Python 3.6 Kernel  </li>
<li> Added libraries are opencv-python==4.0.0.21, seaborn==0.9.0, cuzcatlan==0.9.3, and humanfriendly==4.12.1</li>
</ul>
</div>

In [1]:
from nbtools import UIOutput
uio = UIOutput(description='Checking if all requirements are satisfied. Particulally, checking if opencv-python==4.0.0.21, seaborn==0.9.0, cuzcatlan==0.9.3, and humanfriendly==4.12.1 are installed')
display(uio)


try:
    uio.status = 'Checking...'
    import subprocess
    from IPython.core.display import HTML
    from IPython.display import Javascript
    import cv2
    import seaborn as sns
    from cuzcatlan import add_stat_annotation
    import humanfriendly
    import nbtools
    uio.status = 'Done!'
    
except ModuleNotFoundError:
    uio.status = 'Installing...'
    out = subprocess.run(["pip", "install","--user","-U", "opencv-python==4.0.0.21","seaborn==0.9.0", "cuzcatlan==0.9.3","humanfriendly==4.12.1"])
    if out.returncode == 0:
        print("Successfully installed opencv-python==4.0.0.21, seaborn==0.9.0, and cuzcatlan==0.9.3")
        HTML("<script>Jupyter.notebook.kernel.restart()</script>")
        display(Javascript('require("notebook/js/notebook").Notebook.prototype.scroll_to_bottom = function () {}'))
        display(Javascript('IPython.notebook.execute_all_cells()'))
        
        uio.status = 'Done!'
    else:
        uio.status = 'Unexected error.'
        print(out)
except ImportError:
    uio.status = 'Installing...'
    out = subprocess.run(["pip", "install","--user","-U", "opencv-python==4.0.0.21","seaborn==0.9.0", "cuzcatlan==0.9.3","humanfriendly==4.12.1"])
    if out.returncode == 0:
        print("Successfully installed opencv-python==4.0.0.21, seaborn==0.9.0, and cuzcatlan==0.9.3")
        HTML("<script>Jupyter.notebook.kernel.restart()</script>")
        display(Javascript('require("notebook/js/notebook").Notebook.prototype.scroll_to_bottom = function () {}'))
        display(Javascript('IPython.notebook.execute_all_cells()'))
        uio.status = 'Done!'
    else:
        uio.status = 'Unexected error.'
        print(out)

UIOutput(description='Checking if all requirements are satisfied. Particulally, checking if opencv-python==4.0…

In [2]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

# Analyses

## Step 1: Find cells on control image

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>

- Upload the negative control image (i.e., the one where the migration region is most clearly visible). For the example choose 'MDA231_stopper_1_c3.tif' from the dropdown menu.<br>
    
- Optionally, change <b>kernel size</b> and <b>pixel threshold</b> to iteratively improve the creation of the cells mask.
</div>

In [3]:
%matplotlib inline
# following the floodfill algorithm described in https://www.learnopencv.com/filling-holes-in-an-image-using-opencv-python-c/
import cv2
import numpy as np
from skimage import draw
from skimage import io
import matplotlib.pyplot as plt
from scipy import optimize
import humanfriendly
from timeit import default_timer as timer
import os
import pandas as pd
import seaborn as sns
from cuzcatlan import add_stat_annotation
import subprocess
import validators
from urllib.parse import urlparse

def cost(params):
    global im2
    maxy, maxx = im2.shape
    maxr = min(maxx,maxy)/2
    area = maxy*maxx
    
    x0= params[0]
    y0 = params[1]
    r0 = params[2]
    
    coords = draw.circle(y0, x0, r0, shape=im2.shape)
    template = np.zeros_like(im2) #set all values to be zero
    template[coords] = 1
    
    mask_size = np.sum(template)
    cell_pixels_covered_by_mask = np.sum(template&im2)
    penalty_harshness = 10
    
    score = mask_size - penalty_harshness*cell_pixels_covered_by_mask
    score = score/area
        
    return -score

def download(url):
    filename = os.path.basename(urlparse(url).path)
    if not os.path.isfile(filename):
        subprocess.check_output(['wget',url])
        print(f'The file named "{filename}" was downloaded from the url.')
    return filename

setup = {}
@genepattern.build_ui(parameters={
    "control": {        
        "type": "file",
        "kinds": ["tif", "png",'jpg'],
        "description":'The image where the migration region will be identified.',
        "choices":{"MDA231_stopper_1_c3.tif":"https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_stopper_1_c3.tif"},
    },
    "setup": {
        "default": setup,
        "hide":True,
    },
    "kernel_size": {
        "type":'number',
        "default": 2,
        "hide":False,
        "description":'The smaller, the finer the mask. Typically the default (2) is good enough.',
    },
    "pixel_threshold": {
        "type":'number',
        "default": 20,
        "hide":False,
        "description":'The pixel intensity value to clasify a cell to be black (0) or white (255). Typically the default (20) is good enough.',
    },
    
    "output_var": {
        "default": "setup",
        "description": "The results of the function, must be called 'setup'",
        "hide": True,
    }
})
def create_mask(control,kernel_size=2, pixel_threshold=20,setup='setup'):
    setup = {}
    beginning_of_time = timer()
    if validators.url(control):
        control = download(control) 
    # Read image
    im_in = cv2.imread(control, cv2.IMREAD_GRAYSCALE)
    
    if pixel_threshold > 255:
        pixel_threshold = 255
    elif pixel_threshold < 0:
        pixel_threshold = 0

    # Threshold. ==> These could be parameters
    # Set values equal to or above 20 to 0.
    # Set values below 20 to 255.
    th, im_th = cv2.threshold(im_in, 20, 255, cv2.THRESH_BINARY_INV)

    # Copy the thresholded image.
    im_floodfill = im_th.copy()

    # Mask used to flood filling.
    # Notice the size needs to be 2 pixels than the image.
    h, w = im_th.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    # Floodfill from point (0, 0)
    cv2.floodFill(im_floodfill, mask, (0,0), 255);

    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)

    # Combine the two images to get the foreground.
    im_out = im_th | im_floodfill_inv
    io.imsave(fname='temp_output.png', arr=im_out)

    # im_out_inv = cv2.bitwise_not(im_out)


    # dilate the mask:
    k_size = kernel_size
    k_half = k_size/2
    kernel = np.ones((k_size,k_size),np.uint8)
    coords = draw.circle(k_half, k_half, k_half, shape=im_th.shape)
    kernel[coords] = 1 
    erosion = cv2.erode(im_out,kernel,iterations = 1)
    dilation = cv2.dilate(cv2.bitwise_not(erosion),kernel,iterations = 1)
    # cells_mask = cv2.bitwise_not(dilation)
    cells_mask = dilation/255
    
    setup['control_grayscale'] = im_in
    setup['mask'] = cells_mask


    io.imshow(cells_mask)
    plt.show()
    
    print("Note that a value of ~1 means that pixel belongs to the mask and it is rendered as white.")
    print("A value of 0 means it deos not belong the mask and it is rendered as black.")
    end_of_time = timer()
    spanned = end_of_time - beginning_of_time
    print(f"\nDone with this part of the workflow. Elapsed time: {humanfriendly.format_timespan(spanned)}.")
    return setup
    

UIBuilder(function_import='create_mask', name='create_mask', origin='Notebook', params=[{'name': 'control', 'l…

##  Step 2: Find migration region

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>

- Run the cell below to find the migration region, adjust the finesse parameter if you are not satisfied with the results.
</div>

In [4]:
@genepattern.build_ui(
    parameters={
    "setup": {
        "default": setup,
        "hide":True,
    },
    "finesse": {
        "default":6,
        "hide":False,
        "description":"How carefully to choose the migration region. Higher numbers will yield more accurate results at the cost of much more runtime.",
    },
    "output_var": {
        "default": "setup",
        "description": "The results of the function, must be called 'setup'",
        "hide": True,
    }
    
}
)
def find_migration_region(setup='setup',finesse=20):
    beginning_of_time = timer()
    
    global im2
    im2 = setup['control_grayscale']>0.2
    im2 = im2.astype(int)
    
    maxy, maxx = im2.shape
    minx, miny = (0,0)
    maxr = min(maxx,maxy)/2

    x0 = im2.shape[1]/2
    y0 = im2.shape[0]/2
    r0 = min(im2.shape[1],im2.shape[0])/4
    
    xmid = im2.shape[1]/2
    ymid = im2.shape[0]/2
    rmid = min(xmid,ymid)

    coarse = finesse*1/3

    # do fit, here with leastsq model
    # minner = Minimizer(cost_obj, params)
    x_slice = slice(xmid-x0/4, xmid+x0/4, (x0/2)/coarse)
    y_slice = slice(ymid-x0/4, ymid+x0/4, (y0/2)/coarse)
    r_slice = slice(rmid-x0/4, rmid+x0/4, (r0/2)/finesse)
    rranges = (x_slice,y_slice, r_slice)
    print('About to perform optimization. This would take a few seconds to a few minutes.')

    resbrute = optimize.brute(cost, rranges,full_output=True)

    # result = minner.minimize(method='brute',ranges=rranges)
    # report_fit(result)
    print('############')
    method = 'scipy.brute'
    opt_params = resbrute[0]
    x_opt = opt_params[0]
    y_opt = opt_params[1]
    r_opt = opt_params[2]
    print("Optimal paramters are", [x_opt,y_opt,r_opt])
    print("Units are pixels.")
    f, ax = plt.subplots()
    circle = plt.Circle((x_opt, y_opt), r_opt, alpha = 0.5)
    ax.imshow(im2, cmap='gray', interpolation='nearest')
    ax.add_artist(circle)
    print('############')
    print(f'Method "{method}""\tobjective={cost([x_opt,y_opt,r_opt])}')
    print("The smaller the objective function value, the better (negative numbers are OK).")
    print('############')
    plt.show()
    
    coords = draw.circle(y0, x0, r0, shape=im2.shape)
    template = np.zeros_like(im2) #set all values to be zero
    template[coords] = 1
    
    setup['im2'] = im2
    setup['opt_params'] = opt_params
    setup['x_opt'] = x_opt
    setup['y_opt'] = y_opt
    setup['r_opt'] = r_opt
    setup['circle'] = circle
    setup['coords'] = coords
    setup['template'] = template
    
    end_of_time = timer()
    spanned = end_of_time - beginning_of_time
    print(f"\nDone with this part of the workflow. Elapsed time: {humanfriendly.format_timespan(spanned)}.")
    
    return setup

UIBuilder(function_import='find_migration_region', name='find_migration_region', origin='Notebook', params=[{'…

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>

</div>

## Step 3: Upload files

In this step we will quantify the cell migration on the provided images. Please either upload your images in the cell below or download the ones used in the "Quantification of cancer cell migration with an integrated experimental-computational pipeline " paper by following instructions in the next step ("Optional: reproduce Juarez et al. 2018").

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>

<ol>
    <li>Select how many images you will upload.</li>
    <li>Click <b>Run</b></li>
    <li>For each image, click <b>Upload File...</b> and selec it from your local files</li>
</ol>
<b>Note:</b> Please name your files by groups. For example, all of the files with a stopper should have a unique word associated with them (e.g., stopper), this naming convention will be used in the next step.
</div>

In [11]:
@genepattern.build_ui(parameters={
    "how_many": {
        "type": "number",
        "description":"How many files to upload",
    },
    "output_var": {
        "default": None,
        "hide": True,
    }
})
def upload_files(how_many=2):
    files = ''
    decorator_string = ''
    for i in range(how_many):
        current_file = 'file_'+str(i+1)+', '
        files += current_file
        current_decorator = '"%s": {"type": "file", "kinds": ["tif", "png","jpg"],},' % (current_file.strip(', '))
        decorator_string += current_decorator
    
    func = '''
@genepattern.build_ui(parameters={"output_var": {"default": None,"hide": True,},"setup":{"default": setup,"hide":True,},%s})
def file_example(%s):
    setup['uploaded_files'] = [%s]
    print("All files were uploaded. Move along.")
    return
'''% (decorator_string,files,files)
    get_ipython().run_cell(func) ## This runs the code in the outer scope, so no need to return 'setup'

UIBuilder(function_import='upload_files', name='upload_files', origin='Notebook', params=[{'name': 'how_many',…

### Optional: reproduce Juarez et al. 2018

<div class="alert alert-warning">
<h3 style="margin-top: 0;"> Warning! <i class="fa fa-exclamation-circle"></i></h3>
<b>Only run this step if you are interested in replicating the results from Juarez et al. 2018</b>
</div>

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
Click run to download the files from Juarez et al. 2018 
</div>

In [6]:
@nbtools.build_ui(
description="This function downlowads the images used in Juarez et al. 2018",
parameters={
"setup": {
        "default": "setup",
        "hide":True,
    },
"output_var": {
    "default": "setup",
    "description": "The results of the function, must be called 'setup'",
    "hide": True,
    },
})
def download_files_for_article_reproducibility(setup=setup):
    list_of_images = [
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_stopper_1_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_stopper_2_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_stopper_3_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_stopper_4_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_untreated_1_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_untreated_2_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_untreated_3_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_untreated_4_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_AGR2ab-150_1_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_AGR2ab-150_2_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_AGR2ab-150_3_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_AGR2ab-150_4_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_Taxol10nM_1_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_Taxol10nM_2_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_Taxol10nM_3_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_Taxol10nM_4_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_IgG-150_1_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_IgG-150_2_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_IgG-150_3_c3.tif',
        'https://github.com/edjuaro/cell-migration-quantification/raw/master/images/MDA231_IgG-150_4_c3.tif',
    ]
    downloaded_filenames = []
    for image in sorted(list_of_images):
        file = download(image)
        downloaded_filenames.append(file)
    setup['uploaded_files'] = downloaded_filenames
    print("All files downloaded.")
    return setup

UIBuilder(description='This function downlowads the images used in Juarez et al. 2018', function_import='downl…

## Step 4: Compute and plot migration quantification

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
Make sure you have uploaded the images you will test.
</div>

In [7]:
@nbtools.build_ui(parameters={
    "setup": {
        "default": setup,
        "hide":True,
    },
    "folder": {
        "default":"N/A",
        "description":"The name (including relative path, e.g., 'assays/images/') of the folder where the images are. Leave as 'N/A if you uploaded your files in the previous step.'",
    },
    "conditions": {
        "default":"N/A",
        "description":"The list of conditions by which to group images **separated by a coma and a space** (e.g., 'stopper, untreated, AGR2ab, Taxol, IgG'). These words need to be part of the naming convention.",
    },
    "conditions_to_compare": {
        "default":"N/A",
        "description":"The list of conditions to compare (using the selected statistical text -- Wilcoxon) **pairs separated by a coma and a space; tests separated by semicolon and space** (e.g., 'untreated, AGR2ab; untreated, Taxol; untreated, IgG'). These words need to be part of the naming convention. Leave as 'N/A' for no test.",
    },
    "statistical_test": {
        "default": "Mann-Whitney",
        "type": "choice",
        "choices": {
            "Recomended: Mann-Whitney (Wilcoxon)": "Mann-Whitney",
            "t-test (independent samples)": "t-test_ind",
            "Rarely used: t-test (technical replicates)": "t-test_paired",
        }
    },
    "output_var": {
        "default": None,
        "hide": True,
    }
})
def load_images(conditions,statistical_test = 'wilcoxon', conditions_to_compare='N/A',folder='N/A',setup=setup,verbose=False):
    if 'N/A' in folder:
        all_files = setup['uploaded_files']
    else:
        all_files = sorted(os.listdir(folder))
        ### Add a helpful error message is folder is selected by no images present
    
    filename = []
    condition = []
    percent_covered = []
    
    if isinstance(conditions, str):
        conditions = conditions.split(', ')
    
    if isinstance(conditions_to_compare, str):
        conditions_to_compare = conditions_to_compare.split('; ')
    
    if 'N/A' in conditions_to_compare:
        to_test = []
    else:
        to_test = []
        for temp in conditions_to_compare:
            temp_tuple = tuple(temp.split(', '))
            to_test.append(temp_tuple)
      
    for category in conditions:
        curr_files = [i for i in all_files if category in i]
        if verbose:
            print(category,curr_files)
        for image in curr_files:
            if verbose:
                print(f"\tWorking with {image}")
            if 'N/A' in folder:
                current_filename = image
            else:
                current_filename = os.path.join(folder,image)
            im = io.imread(current_filename,as_gray=True)
            im01 = im>0
            im01 = im01.astype(int)
            if False:
                f, ax = plt.subplots()
                ax.imshow(im01, cmap='gray')
                circle = plt.Circle((setup['x_opt'], setup['y_opt']), setup['r_opt'], alpha = 0.5)
                ax.add_artist(circle)
                plt.show()
            
            # create the mask on top of this image
            coords = draw.circle(setup['y_opt'], setup['x_opt'], setup['r_opt'], shape=im01.shape)
            template = np.zeros_like(im01) #set all values to be zero
            template[coords] = 1
            cell_pixels_covered_by_mask = np.sum(template&im01)
#             print(100*cell_pixels_covered_by_mask/np.sum(template))
            filename.append(image)
            condition.append(category)
            percent_covered.append(100*cell_pixels_covered_by_mask/np.sum(template))
            
    df = pd.DataFrame({"condition": condition, "percent_covered": percent_covered, "filename" : filename})


    f, ax = plt.subplots(figsize=(16,9))
    ax=sns.barplot(x="condition", y="percent_covered", data=df, dodge=1, ax=ax, ci=None)
    ax=sns.stripplot(x="condition", y="percent_covered", data=df, ax=ax, linewidth=2, edgecolor='gray')
    if to_test:
        add_stat_annotation(ax, data=df, x='condition', y='percent_covered',
                            boxPairList=to_test,
                            test=statistical_test, textFormat='star', loc='inside', verbose=2)
    return 

UIBuilder(function_import='load_images', name='load_images', origin='Notebook', params=[{'name': 'conditions',…